In [3]:
%pip install yfinance
%pip install tqdm

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/14/03/3c854ca3e02eedf614abba4b2e177c469bf3af58207fa30d5098c5d652fe/yfinance-0.2.37-py2.py3-none-any.whl.metadata
  Using cached yfinance-0.2.37-py2.py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for pandas>=1.3.0 from https://files.pythonhosted.org/packages/71/00/6beaeeba7f075d15ea167a5caa039b861e58ff2f58a5b659abb9b544c8f6/pandas-2.2.1-cp312-cp312-win_amd64.whl.metadata
  Using cached pandas-2.2.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Obtaining dependency information for numpy>=1.16.5 from https://files.pythonhosted.org/packages/16/2e/86f24451c2d530c88daf997cb8d6ac622c1d40d19f5a031ed68a4b73a374/numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Obtaining dependency information for requests>=2.31 from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b4

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: c:\Users\Owner\.pyenv\pyenv-win\versions\3.12.1\python.exe -m pip install --upgrade pip


  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/2a/14/e75e52d521442e2fcc9f1df3c5e456aead034203d4797867980de558ab34/tqdm-4.66.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.6 kB 330.3 kB/s eta 0:00:01
     --------------------------------- ---- 51.2/57.6 kB 375.8 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 378.8 kB/s eta 0:00:00
Using cached tqdm-4.66.2-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: c:\Users\Owner\.pyenv\pyenv-win\versions\3.12.1\python.exe -m pip install --upgrade pip


In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import os
import json
from tqdm import tqdm

In [5]:
def fetch_sp500_daily_data():
    sp500_data = yf.download('^GSPC', start='2017-12-20', end='2018-06-10')
    return sp500_data

# Example usage
sp500_data = fetch_sp500_daily_data()
sp500_data = pd.DataFrame(sp500_data)

# Create a new dataframe to store the result
sp500_windows = pd.DataFrame()

# Iterate through each date in the dataframe
for date in sp500_data.index:
    close_values = []
    
    for i in range(-10, 11):
        offset_date = date + pd.DateOffset(days=i)
        if offset_date in sp500_data.index:
            close_value = sp500_data.loc[offset_date, 'Close']
        else:
            close_value = float('nan')
        close_values.append(close_value)
    
    result_row = {'Date': date}
    for i, close_value in zip(range(-10, 11), close_values):
        result_row[f'{i}'] = close_value

    sp500_windows = sp500_windows._append(result_row, ignore_index=True)

[*********************100%%**********************]  1 of 1 completed


In [15]:
# List of directories containing JSON files
directories = ['2018_01', '2018_02', '2018_03', '2018_04', '2018_05']  # Add all your directory paths here

# List to store extracted data
data_list = []

# Loop through directories
for directory in tqdm(directories):
    # List of JSON files in the directory
    json_files = [file for file in os.listdir(directory) if file.endswith('.json')]
        
    # Loop through JSON files in the directory
    for file_name in tqdm(json_files):
        # Read JSON data from file
        with open(os.path.join(directory, file_name), 'r', encoding='utf-8') as f:
            json_data = f.read()
        
        data = json.loads(json_data)

        data_list.append({
            'text': data['text'],
            'published': data['thread']['published'],
            'title': data['thread']['title']
        })

100%|██████████| 5/5 [21:30<00:00, 258.18s/it]


In [18]:
# Create DataFrame
news_articles = pd.DataFrame(data_list)
news_articles['published'] = pd.to_datetime(news_articles['published'], utc=True)
news_articles['published'] = news_articles['published'].dt.tz_convert(None)

# Merging the dataframes based on 'published' and 'date' columns
merged_df = pd.merge(news_articles, sp500_windows, left_on='published', right_on='Date', how='inner')

# Dropping the 'date' column as it's redundant
merged_df.drop(columns=['published'], inplace=True)


In [107]:
# Create DataFrame from data_list
news_articles = pd.DataFrame(data_list)

# Convert 'published' column to datetime with timezone information
news_articles['published'] = pd.to_datetime(news_articles['published'], utc=True)
news_articles['published'] = news_articles['published'].dt.tz_convert(None)
news_articles['published'] = pd.to_datetime(news_articles['published'].dt.date)

# Merging the dataframes based on 'published' and 'Date' columns with a left join
merged_df = pd.merge(news_articles, sp500_windows, left_on='published', right_on='Date', how='left')

new_dates = set()

# Calculate the date range for stock prices (-10 to 10 days)
for _, row in tqdm(merged_df.iterrows()):

    # Check if the value in column '0' is NaN
    if pd.isna(row['0']):
        date = row['published']
        new_dates.add(date)
    
        for i in range(-10, 11):
            offset_date = date + pd.DateOffset(days=i)
            if offset_date in sp500_data.index:
                row[f'{i}'] = sp500_data.loc[offset_date, 'Close']
            else:
                row[f'{i}'] = float('nan')

# Dropping the 'Date' column as it's redundant
merged_df.drop(columns=['Date'], inplace=True)

306242it [00:52, 5842.52it/s] 


In [108]:
# Initialize a new column 'percent_change' with NaN values
merged_df['percent_change'] = np.nan
merged_df['change'] = np.nan

# Loop through each row
for index, row in merged_df.iterrows():
    # Find the index of the next non-null value to the left of column '0'
    next_non_null_index = None
    for i in range(1, 11):
        if row[str(i)] is not np.nan:
            next_non_null_index = str(i)
            break
    
    prev_non_null_index = None
    for i in range(11):
        if row[str(-i)] is not np.nan:
            prev_non_null_index = str(-i)
            break

    # If next non-null value exists and it's not the first column
    if prev_non_null_index and next_non_null_index:
        # Calculate the percentage change between the value in column '0' and the next non-null value
        change = (row[prev_non_null_index] / row[next_non_null_index]) - 1
        merged_df.at[index, 'percent_change'] = change
        if change > 0:
            merged_df.at[index, 'change'] = 1
        else:
            merged_df.at[index, 'change'] = 0


In [125]:
merged_df = merged_df.dropna(subset=['percent_change'])
merged_df = merged_df.dropna(subset=['text'])

In [126]:
merged_df

,text,published,title,-10,-9,-8,-7,-6,-5,-4,...,3,4,5,6,7,8,9,10,percent_change,change
0,17 Hours Ago | 02:56 \nEmerging markets soared...,2018-01-03,Emerging markets are set for an even bigger ra...,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,NaN,...,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.239990,NaN,-0.004012,0.0
1,Cramer reflects on how Trump's actions are fue...,2018-01-02,Cramer reflects on how Trump's actions are fue...,NaN,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,...,2743.149902,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.23999,-0.006358,0.0
2,"Published: Jan 2, 2018 5:59 p.m. ET Share \nFe...",2018-01-02,The Wall Street Journal: Peter Thiel’s VC firm...,NaN,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,...,2743.149902,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.23999,-0.006358,0.0
3,By Reuters 8:23 AM EST \nTelevision host Hoda ...,2018-01-02,Hoda Kotb Will Replace Matt Lauer on NBC’s ‘To...,NaN,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,...,2743.149902,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.23999,-0.006358,0.0
5,"January 3, 2018 / 12:54 AM / Updated 14 hours ...",2018-01-03,U.S. judge orders government to release Iraqis...,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,NaN,...,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.239990,NaN,-0.004012,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306237,"CHICAGO, May 30, 2018 (GLOBE NEWSWIRE) -- Dres...",2018-05-30,Ejaz Elahi Rejoins Dresner Partners as Managin...,NaN,2733.010010,2724.439941,2733.290039,2727.760010,2721.330078,NaN,...,NaN,NaN,2746.870117,2748.800049,2772.350098,2770.370117,2779.030029,NaN,0.006927,1.0
306238,"May 31, 2018 / 7:59 AM / Updated 2 minutes ago...",2018-05-31,UPDATE 1-Japan Post Insurance ready to buy sho...,2733.01001,2724.439941,2733.290039,2727.760010,2721.330078,NaN,NaN,...,NaN,2746.870117,2748.800049,2772.350098,2770.370117,2779.030029,NaN,NaN,-0.010733,0.0
306239,"SHANGHAI, China, May 31, 2018 (GLOBE NEWSWIRE)...",2018-05-31,GDS Holdings Limited Prices Offering of Conver...,2733.01001,2724.439941,2733.290039,2727.760010,2721.330078,NaN,NaN,...,NaN,2746.870117,2748.800049,2772.350098,2770.370117,2779.030029,NaN,NaN,-0.010733,0.0
306240,Nathan Eovaldi threw six no-hit innings in his...,2018-05-31,Major League Baseball roundup: Rays' Eovaldi d...,2733.01001,2724.439941,2733.290039,2727.760010,2721.330078,NaN,NaN,...,NaN,2746.870117,2748.800049,2772.350098,2770.370117,2779.030029,NaN,NaN,-0.010733,0.0


In [127]:
# Write to CSV
merged_df.to_csv('news_and_stocks.csv', index=False)

In [124]:
merged_df[merged_df['published'] == '2018-01-01']
merged_df[pd.isna(merged_df['percent_change'])]

,text,published,title,-10,-9,-8,-7,-6,-5,-4,...,3,4,5,6,7,8,9,10,percent_change,change
4,"January 1, 2018 / 8:56 AM / Updated 5 hours ag...",2018-01-01,UK's Compass says new CEO to start Jan 1 after...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
10,Bitcoin is already having a bad year.\nFor the...,2018-01-01,Bitcoin Is Already Having a Bad Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
12,"Retail American Girl Debuts New Doll, an Aspir...",2018-01-01,"American Girl Debuts New Doll, an Aspiring Ast...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
25,Copper prices set to continue gains in 2018 20...,2018-01-01,Copper prices set to continue gains in 2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
39,There's a risk of 'collision of intent' on Nor...,2018-01-01,There's a risk of 'collision of intent' on Nor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305732,May 28 (Reuters) - Centrais Eletricas Brasilei...,2018-05-28,BRIEF-Brazil's Eletrobras says board approved ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
305944,* STOXX 600 down 0.3 pct\n* Italian banks suff...,2018-05-28,UPDATE 1-Italian stocks' recovery short-lived ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
305945,"LONDON, May 28 (Reuters) - Italian bonds, stoc...",2018-05-28,Italian markets rally as eurosceptic coalition...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
305946,Venture Capital A Chinese bank chairman was fo...,2018-05-28,Chinese bank chief found dead inside his office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [123]:
set1 = set(new_dates)
set2 = set(merged_df[pd.isna(merged_df['percent_change'])]['published'].unique())
print(len(set1))
print(len(set2))
len(set1.intersection(set2))

56
79


56

In [100]:
sp500_windows

,Date,-10,-9,-8,-7,-6,-5,-4,-3,-2,...,1,2,3,4,5,6,7,8,9,10
0,2017-12-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2684.570068,2683.340088,NaN,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,NaN
1,2017-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2683.340088,NaN,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,NaN,NaN
2,2017-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2679.250000,...,NaN,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,NaN,NaN,NaN
3,2017-12-26,NaN,NaN,NaN,NaN,2679.250000,2684.570068,2683.340088,NaN,NaN,...,2682.620117,2687.540039,2673.610107,NaN,NaN,NaN,2695.810059,2713.060059,2723.989990,2743.149902
4,2017-12-27,NaN,NaN,NaN,2679.250000,2684.570068,2683.340088,NaN,NaN,NaN,...,2687.540039,2673.610107,NaN,NaN,NaN,2695.810059,2713.060059,2723.989990,2743.149902,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,2018-06-04,2721.330078,NaN,NaN,NaN,2689.860107,2724.010010,2705.270020,2734.620117,NaN,...,2748.800049,2772.350098,2770.370117,2779.030029,NaN,NaN,NaN,NaN,NaN,NaN
113,2018-06-05,NaN,NaN,NaN,2689.860107,2724.010010,2705.270020,2734.620117,NaN,NaN,...,2772.350098,2770.370117,2779.030029,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,2018-06-06,NaN,NaN,2689.860107,2724.010010,2705.270020,2734.620117,NaN,NaN,2746.870117,...,2770.370117,2779.030029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,2018-06-07,NaN,2689.860107,2724.010010,2705.270020,2734.620117,NaN,NaN,2746.870117,2748.800049,...,2779.030029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
test = pd.read_csv('news_and_stocks.csv')
test

,text,published,title,-10,-9,-8,-7,-6,-5,-4,...,3,4,5,6,7,8,9,10,percent_change,change
0,17 Hours Ago | 02:56 \nEmerging markets soared...,2018-01-03,Emerging markets are set for an even bigger ra...,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,NaN,...,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.239990,NaN,-0.004012,0.0
1,Cramer reflects on how Trump's actions are fue...,2018-01-02,Cramer reflects on how Trump's actions are fue...,NaN,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,...,2743.149902,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.23999,-0.006358,0.0
2,"Published: Jan 2, 2018 5:59 p.m. ET Share \nFe...",2018-01-02,The Wall Street Journal: Peter Thiel’s VC firm...,NaN,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,...,2743.149902,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.23999,-0.006358,0.0
3,By Reuters 8:23 AM EST \nTelevision host Hoda ...,2018-01-02,Hoda Kotb Will Replace Matt Lauer on NBC’s ‘To...,NaN,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,...,2743.149902,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.23999,-0.006358,0.0
4,"January 3, 2018 / 12:54 AM / Updated 14 hours ...",2018-01-03,U.S. judge orders government to release Iraqis...,NaN,NaN,2680.500000,2682.620117,2687.540039,2673.610107,NaN,...,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.239990,NaN,-0.004012,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228383,"CHICAGO, May 30, 2018 (GLOBE NEWSWIRE) -- Dres...",2018-05-30,Ejaz Elahi Rejoins Dresner Partners as Managin...,NaN,2733.010010,2724.439941,2733.290039,2727.760010,2721.330078,NaN,...,NaN,NaN,2746.870117,2748.800049,2772.350098,2770.370117,2779.030029,NaN,0.006927,1.0
228384,"May 31, 2018 / 7:59 AM / Updated 2 minutes ago...",2018-05-31,UPDATE 1-Japan Post Insurance ready to buy sho...,2733.01001,2724.439941,2733.290039,2727.760010,2721.330078,NaN,NaN,...,NaN,2746.870117,2748.800049,2772.350098,2770.370117,2779.030029,NaN,NaN,-0.010733,0.0
228385,"SHANGHAI, China, May 31, 2018 (GLOBE NEWSWIRE)...",2018-05-31,GDS Holdings Limited Prices Offering of Conver...,2733.01001,2724.439941,2733.290039,2727.760010,2721.330078,NaN,NaN,...,NaN,2746.870117,2748.800049,2772.350098,2770.370117,2779.030029,NaN,NaN,-0.010733,0.0
228386,Nathan Eovaldi threw six no-hit innings in his...,2018-05-31,Major League Baseball roundup: Rays' Eovaldi d...,2733.01001,2724.439941,2733.290039,2727.760010,2721.330078,NaN,NaN,...,NaN,2746.870117,2748.800049,2772.350098,2770.370117,2779.030029,NaN,NaN,-0.010733,0.0
